# Assignment 5 · Revisiting HHL for a 4×4 Linear System
We rebuild the workflow for simulating the Harrow–Hassidim–Lloyd (HHL) quantum linear solver on a two-qubit system, compare it with a classical baseline, and assess when the quantum routine offers value.

This refreshed notebook mirrors the structured workflow from Assignment 3 so every phase stays auditable and reproducible.

**Task outline**
1. Summarise why fast quantum linear solvers support calibration or simulation pipelines and document the runtime stack.
2. Specify a Hermitian 4×4 system, normalise the right-hand side, and record classical diagnostics.
3. Execute HHL with the latest Qiskit primitives, align the quantum amplitudes with the classical baseline, and log comparison metrics.
4. Feed the HHL statevector through the previously defined tomography surrogate to reconstruct the solution and analyse reconstruction quality.
5. Interpret the combined metrics, highlight QST’s role in HHL workflows, and capture takeaways.

## Background notes
- HHL targets systems $A\vec{x}=\vec{b}$ where $A$ is Hermitian and sized $2^n \times 2^n$, embedding the matrix into a unitary, applying phase estimation, and inverting eigenvalues by a controlled rotation.
- We choose a modest 4×4 Hermitian matrix with a friendly condition number so that simulation resources focus on algorithmic steps rather than numerical instability.
- Diagnostics include component-wise differences, the $\ell_2$ vector error, and the residual norm $\lVert A\vec{x}_{\text{est}}-\vec{b}\rVert_2$ to keep classical and quantum results comparable.

## Task 1 · Environment setup


In [ ]:
# Task 1 · Environment check
!pip install --quiet "qiskit==0.46.0" "qiskit-terra==0.46.0" "qiskit-algorithms==0.3.1"

In [ ]:
# Supporting numerical stack (compatible with Qiskit 0.46.x)
!pip install --quiet "numpy<2" "scipy" "pandas" "matplotlib"

In [ ]:
import importlib.metadata as metadata

packages = ["qiskit", "qiskit-algorithms", "numpy", "scipy", "pandas", "matplotlib"]
for name in packages:
    try:
        print(f"{name}: {metadata.version(name)}")
    except metadata.PackageNotFoundError:
        print(f"{name}: not installed")

## Task 2 · Specify the linear system
- Construct a 4×4 Hermitian matrix with moderate off-diagonal coupling so the eigenstructure remains well-conditioned.
- Prepare a right-hand-side vector, normalise it for quantum encoding, and capture the classical baseline solution along with matrix diagnostics.
- Define `A` and `b`, assert Hermiticity, and compute eigenvalues plus the condition number.
- Solve the system classically, store normalised vectors, and package diagnostics for later comparison.

In [ ]:
import numpy as np
import pandas as pd

# Hermitian 4×4 matrix with tridiagonal coupling
A = np.array([[1.0, -0.25, 0.0, 0.0],
              [-0.25, 1.1, -0.25, 0.0],
              [0.0, -0.25, 1.1, -0.25],
              [0.0, 0.0, -0.25, 1.0]], dtype=np.complex128)

if not np.allclose(A, A.conj().T):
    raise ValueError("Matrix A must be Hermitian for HHL; adjust the coefficients.")

b = np.array([1.0, 0.6, 0.4, 0.0], dtype=np.complex128)
b_norm = b / np.linalg.norm(b)

eigenvalues = np.linalg.eigvalsh(A)
condition_number = np.linalg.cond(A)

x_classical = np.linalg.solve(A, b)
x_classical_norm = x_classical / np.linalg.norm(x_classical)
baseline_residual = np.linalg.norm(A @ x_classical - b)

system_df = pd.DataFrame({
    "component": ["x0", "x1", "x2", "x3"],
    "b": b,
    "b_normalised": b_norm,
    "classical_solution": x_classical,
    "classical_normalised": x_classical_norm
})

diagnostics = {
    "matrix_condition_number": float(condition_number),
    "baseline_residual_norm": float(baseline_residual),
    "eigenvalue_min": float(eigenvalues.min()),
    "eigenvalue_max": float(eigenvalues.max()),
    "rhs_norm": float(np.linalg.norm(b))
}

system_df, diagnostics

## Task 3 · Implement the HHL solver
- Build a `LinearSystemProblem` from the normalised right-hand side and run HHL using the default statevector sampler bundled with modern Qiskit (no external Aer dependency required).
- Recover the solution amplitudes, rescale them onto the classical reference, and compute error metrics.

- Implement a helper that executes HHL and extracts the primary solution register.
- Align the quantum output with the classical solution, then tabulate component-wise errors and aggregated metrics.

In [ ]:
from qiskit.quantum_info import Statevector
import numpy as np
import pandas as pd

def run_hhl_and_extract(A: np.ndarray, b_normalised: np.ndarray):
    """
    TODO: Implement the HHL algorithm to solve the linear system Ax=b.

    This function should:
    1. Define a `LinearSystemProblem`.
    2. Instantiate and run the `HHL` solver.
    3. Extract the full statevector from the result.
    4. Isolate the raw solution amplitudes from the appropriate register.
    5. Normalise the solution vector.
    6. Return the raw solution, normalised solution, full statevector, and the HHL result object.
    """
    pass


In [ ]:
raw_hhl, norm_hhl, full_statevector, hhl_result = run_hhl_and_extract(A, b_norm)

# Align global phase and amplitude with the classical reference
scale_factor = np.vdot(x_classical, norm_hhl) / np.vdot(norm_hhl, norm_hhl)
x_hhl_rescaled = scale_factor * norm_hhl

l2_error = np.linalg.norm(x_hhl_rescaled - x_classical)
relative_error = l2_error / np.linalg.norm(x_classical)
residual = np.linalg.norm(A @ x_hhl_rescaled - b)

comparison_df = pd.DataFrame({
    "component": ["x0", "x1", "x2", "x3"],
    "classical": x_classical,
    "hhl_rescaled": x_hhl_rescaled,
    "abs_error": np.abs(x_hhl_rescaled - x_classical)
})

metrics = {
    "l2_error": float(l2_error),
    "relative_error": float(relative_error),
    "residual_norm": float(residual),
    "scale_factor": complex(scale_factor)
}

comparison_df, metrics

## Task 4 · Execute HHL and compare solutions
- Run the HHL solver on the prepared linear system.
- Align the quantum output with the classical solution, then tabulate component-wise errors and aggregated metrics.

## Task 5 · Tomography cross-check with the ML surrogate
- Reuse the quantum state tomography (QST) regression model from Assignment 3 to rebuild the HHL solution from synthetic measurement statistics.
- Generate Pauli-basis expectation values from the HHL statevector, feed them through the surrogate, and recover an estimated statevector.
- Compare the surrogate reconstruction with both the raw HHL amplitudes and the classical baseline to quantify reconstruction accuracy.

**What to do**
- Instantiate the Pauli-basis surrogate, compute expectation values for every measurement setting, and reconstruct the density matrix.
- Extract the principal eigenstate, fix global phase, and report fidelities plus residuals alongside both baselines.

In [ ]:
from itertools import product

PAULI_SINGLE = {
    "I": np.array([[1, 0], [0, 1]], dtype=np.complex128),
    "X": np.array([[0, 1], [1, 0]], dtype=np.complex128),
    "Y": np.array([[0, -1j], [1j, 0]], dtype=np.complex128),
    "Z": np.array([[1, 0], [0, -1]], dtype=np.complex128),
}

def kron_pauli(label: str) -> np.ndarray:
    """Build a multi-qubit Pauli operator from a label such as 'XZ'."""
    op = PAULI_SINGLE[label[0]]
    for char in label[1:]:
        op = np.kron(op, PAULI_SINGLE[char])
    return op

class PauliTomographyModel:
    """Linear Pauli-basis regression surrogate carried over from the QST assignment."""

    def __init__(self, n_qubits: int):
        self.n_qubits = n_qubits
        self.dim = 2 ** n_qubits
        self.labels = ["".join(chars) for chars in product("IXYZ", repeat=n_qubits)]
        self.operators = {label: kron_pauli(label) for label in self.labels}

    def expectation_map(self, statevector: np.ndarray) -> dict[str, float]:
        """Return noiseless Pauli expectation values for the provided statevector."""
        expectations: dict[str, float] = {}
        for label, operator in self.operators.items():
            value = np.vdot(statevector, operator @ statevector)
            expectations[label] = float(np.real_if_close(value))
        return expectations

    def reconstruct_density(self, expectations: dict[str, float]) -> np.ndarray:
        """Rebuild the density matrix via the Pauli expansion ρ = 1/2^n Σ ⟨P⟩ P."""
        rho = np.zeros((self.dim, self.dim), dtype=np.complex128)
        for label, value in expectations.items():
            rho += value * self.operators[label]
        rho /= (2 ** self.n_qubits)
        # Enforce Hermiticity and trace normalisation to counter numerical noise
        rho = 0.5 * (rho + rho.conj().T)
        trace = np.trace(rho)
        if not np.isclose(trace, 1.0):
            rho /= trace
        return rho

    def estimate_statevector(self, expectations: dict[str, float]) -> tuple[np.ndarray, np.ndarray, float]:
        """Return the principal eigenstate reconstructed from the tomography density matrix."""
        rho = self.reconstruct_density(expectations)
        eigenvalues, eigenvectors = np.linalg.eigh(rho)
        principal_index = int(np.argmax(eigenvalues.real))
        state_estimate = eigenvectors[:, principal_index]
        state_estimate /= np.linalg.norm(state_estimate)
        fidelity = float(np.real_if_close(eigenvalues[principal_index]))
        return state_estimate, rho, fidelity

In [ ]:
# Reconstruct the HHL solution via tomography and compare with baselines
qst_model = PauliTomographyModel(n_qubits=2)

# We need to extract the relevant part of the statevector for tomography
# The HHL circuit has ancillary qubits. The solution is on the first n qubits.
num_solution_qubits = int(np.log2(len(b_norm)))
num_qubits_in_state = full_statevector.num_qubits
solution_state_indices = [i for i in range(2**num_qubits_in_state) if (i >> num_solution_qubits) == 0]
hhl_solution_statevector = full_statevector.data[solution_state_indices]
hhl_solution_statevector /= np.linalg.norm(hhl_solution_statevector)


pauli_expectations = qst_model.expectation_map(hhl_solution_statevector)
qst_state, qst_density, dominant_eigenvalue = qst_model.estimate_statevector(pauli_expectations)

# Align global phase to the direct HHL state for fair comparison
phase_alignment = np.vdot(norm_hhl, qst_state)
if np.abs(phase_alignment) > 0:
    qst_state *= np.exp(-1j * np.angle(phase_alignment))

fidelity_hhl_vs_qst = float(np.abs(np.vdot(norm_hhl, qst_state)) ** 2)
fidelity_classical_vs_qst = float(np.abs(np.vdot(x_classical_norm, qst_state)) ** 2)

x_qst_rescaled = scale_factor * qst_state
l2_error_qst = np.linalg.norm(x_qst_rescaled - x_classical)
relative_error_qst = l2_error_qst / np.linalg.norm(x_classical)
residual_qst = np.linalg.norm(A @ x_qst_rescaled - b)

comparison_qst_df = pd.DataFrame({
    "component": ["x0", "x1", "x2", "x3"],
    "classical": x_classical,
    "hhl_rescaled": x_hhl_rescaled,
    "qst_rescaled": x_qst_rescaled,
    "abs_error_qst": np.abs(x_qst_rescaled - x_classical)
})

tomography_metrics = {
    "fidelity_hhl_vs_qst": fidelity_hhl_vs_qst,
    "fidelity_classical_vs_qst": fidelity_classical_vs_qst,
    "l2_error_qst": float(l2_error_qst),
    "relative_error_qst": float(relative_error_qst),
    "residual_norm_qst": float(residual_qst),
    "dominant_eigenvalue": float(dominant_eigenvalue),
    "num_pauli_features": len(pauli_expectations)
}

comparison_qst_df, tomography_metrics

### Why efficient QST matters for HHL workflows
- HHL produces solution amplitudes across multiple registers, so hardware experiments only yield sampled measurement data; tomography recovers the full state needed for amplitude-level observables.
- Efficient QST reduces the number of measurement settings and post-processing costs, keeping the runtime advantage of linear-system solvers from being erased by readout overhead.
- ML-based surrogates let us amortise reconstruction across many runs (e.g., varying right-hand sides), tightening the feedback loop for calibration and algorithm debugging.

## Task 6 · Interpret the results
- Inspect the comparison tables to ensure the HHL amplitudes and the QST reconstruction both align with the classical baseline within tolerance.
- Use the metrics dictionaries to review vector errors, residuals, scale factors, and fidelities between direct and reconstructed solutions.

**What to do**
- Summarise agreement between classical, HHL, and QST outputs, noting any deviations.
- Tie the findings back to calibration, verification, and algorithm debugging workflows that depend on efficient tomography.

## Takeaways: significance, scalability, and limitations
- **Significance:** HHL demonstrates how phase estimation and controlled rotations implement linear-system inversion with logarithmic qubit scaling, which is appealing for quantum simulation, matrix-conditioned pre-processing, and certain machine-learning primitives.
- **Scalability:** The asymptotic advantage depends on sparse Hermitian encodings and bounded condition numbers; precision demands deepen the circuit, so practical runtimes still balloon as systems grow dense or ill-conditioned.
- **Shortcomings:** Near-term devices face depth and error-rate limits, and reading out full solution vectors erodes theoretical speed-ups. Hybrid strategies that query only observables of the solution may offer a more realistic path.
- **Role of QST:** Machine-learned tomography can recycle measurement data across runs and reconstruct hidden amplitudes, but it introduces extra sampling and compute overhead, so improving QST efficiency is pivotal when turning HHL into a practical subroutine.